Expressões regulares (regex, regexp) são uma linguagem que nos permitem definir padrões que devem ser encontrados em `strings`.

Algumas fontes sobre expressões regulares:
* https://www.regular-expressions.info/
* https://regexr.com/
* https://regex101.com/

Normalmente, a informação é disponibilizada de uma forma estruturada que é fácil para uma pessoa entender.
Entretanto, a mesma informação é difícil de ser estruturada para um computador.

Se a informação possui, de fato, algum tipo de estrutura, é possível estraí-la com uma (ou mais) expressão regular.

Vamos ao exemplo...

In [1]:
# lista de top 10 livros mais vistos no dia anterior no Projeto Gutenberg
# http://www.gutenberg.org/browse/scores/top
top_10 = '''
1. The Works of Edgar Allan Poe, The Raven Edition by Edgar Allan Poe (1525)
2. Pride and Prejudice by Jane Austen (1302)
3. Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley (995)
4. A Modest Proposal by Jonathan Swift (674)
5. Beowulf: An Anglo-Saxon Epic Poem (658)
6. Moby Dick; Or, The Whale by Herman Melville (600)
7. The Strange Case of Dr. Jekyll and Mr. Hyde by Robert Louis Stevenson (560)
8. The Adventures of Sherlock Holmes by Arthur Conan Doyle (547)
9. A Tale of Two Cities by Charles Dickens (491)
10. Ulysses by James Joyce (481)
'''

A lista acima apresenta quatro informações de forma semi estruturada.
Este seria um template básico das informações:
    
`<posição>. <nome da obra> by <nome do autor> (acessos)`

Dada essa estrutura, é possível extrair o conteúdo com expressões regulares

In [2]:
# módulo do python para trabalhar com expressões regulares
import re

# para simplificar, separamos as linhas
lista_top_10 = top_10.splitlines()
lista_top_10 = lista_top_10[1:]

In [3]:
# extrair a <posição> no começo da linha
match = re.search('^\d', lista_top_10[0])
print('Posição:', match.group(0))
print('Posição:', match[0])

Posição: 1
Posição: 1


Na notação de epressão regular, `\d` significa "Qualquer número", ou seja, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9.
Note que é apenas um número apenas, se precisar de dois números, deve ser `\d\d`.

O caractére `^` indica que deve procurar no ínicio da string.

`\d` é uma abreviação para a expressão [0-9] que será demonstrada abaixo

Qual a diferença de `re.search` e `re.match`?

Resposta: ...

Dica: Procure na documentação do python.

In [4]:
# extrair a <posição> do último item da lista
match = re.search('^\d\d', lista_top_10[-1]) # note \d\d
print('Posição:', match.group(0))
print('Posição:', match[0])

Posição: 10
Posição: 10


Entretanto, esse padrão deveria ser flexível para pegar qualquer número até o caractére `.`.
Portanto, é possível usar quantificadores `*`, `+` ou `{n[, m]}`.

* `*`: Zero ou mais ocorrências do padrão.
    * Exemplo: `\d*` captura zero ou mais ocorrências de qualquer número (`''`, `'2'`, `'12'`, `''4565`)
* `+`: Uma ou mais ocorrências do padrão.
    * Exemplo: `\d+` captura zero ou mais ocorrências de qualquer número (`'2'`, `'12'`, `''4565`)
* `{n}`: Exatamente `n` ocorrências do padrão.
    * Exemplo: `\d{2}` captura exatamente dois números seguidos, equivalente à `\d\d` (`'10'`, `'78'`)
* `{n,}`: Ao menos `n` ocorrências do padrão.
    * Exemplo: `\d{2,}` captura ao menos dois números seguidos (`'10'`, `'078'`, `'78912'`)
* `{n,m}`: Ao menos `n` ocorrências do padrão e no máximo `m`.
    * Exemplo: `\d{2,4}` captura ao menos dois números seguidos e no máximo quatro (`'10'`, `'078'`, `'7892'`)

Para o exemplo da posição, sabemos que sempre existirá um número, portanto `+` ou `{1,}` podem ser utilizados.

In [5]:
# extrair a <posição> do último item da lista
match = re.search('^\d+', lista_top_10[-1])
print('Posição:', match.group(0))
print('Posição:', match[0])

match = re.search('^\d+', lista_top_10[0])
print('Posição:', match.group(0))
print('Posição:', match[0])

Posição: 10
Posição: 10
Posição: 1
Posição: 1


Agora é possível pegar o número de visualizações no último dia ao final da string.

Para simplificar, vamos pegar apenas as ocorrências por enquanto, no final juntaremos tudo em uma expressão.

Como é um número, `\d` pode ser utilizado e como sabemos que existe ao menos um número, `+` também pode ser utilizado.

In [6]:
match = re.search('\(\d+\)$', lista_top_10[0])
print('Visualizações:', match.group(0))
print('Visualizações:', match[0])

Visualizações: (1525)
Visualizações: (1525)


Algumas pontos à serem notados:
* `$` foi utilizado para denotar que a busca deve ser feita no final da `string` e não no começo `^`
* `\(` e não apenas `(` para notar o `(` no texto.
    * O caractére `(` faz parte da linguagem de expressão regular, portanto, temos que utilizar o `\` antes.
* Escrever apenas o padrão `\(` faz a busca por esse padrão
* A existência de `()` na saída, mas queremos apenas os números

Vamos explorar um pouco o ponto _Escrever apenas o padrão `\(` faz a busca por esse padrão_

In [7]:
match = re.search('Algo', 'Algo por aqui')
print(match.group(0))
print(match[0])

match = re.search('por', 'Algo por aqui')
print(match.group(0))
print(match[0])

match = re.search('aqui', 'Algo por aqui')
print(match.group(0))
print(match[0])

match = re.search('Al', 'Algo por aqui')
print(match.group(0))
print(match[0])

Algo
Algo
por
por
aqui
aqui
Al
Al


Note que qualquer texto funciona como uma expressão regular.
Porém, eles não são muito úteis diretamente, visto que já sabemos o conteúdo.
O verdadeiro uso é para delimitar partes do texto, por exemplo `()` no exemplo anterior.
Ou `by` para separar o `<nome da obra>` do `<nome do autor>` como veremos adiante.

O outro ponto a ser explorado é como remover os `()` da saída.
Para isso precisamos de grupos, delimitados por `()` (lembra que foi mencionado que `()` fazem parte das expressões regulares?).

Um grupo é simplesmente um padrão entre `()` e ele pode ser referenciado posteriormente.
No exemplo para coletar as visualizações..

In [8]:
match = re.search('\((\d+)\)$', lista_top_10[0]) # note o grupo (\d+) dentro dos delimitadores \(\)
print('Visualizações:', match.group(1)) # note que não é mais 0
print('Visualizações:', match[1]) # note que não é mais 0

Visualizações: 1525
Visualizações: 1525


In [9]:
# também é possível dar nome ao grupo
match = re.search('\((?P<visualizacoes>\d+)\)$', lista_top_10[0]) # note o grupo (\d+) dentro dos delimitadores \(\)
print('Visualizações:', match.group('visualizacoes')) # note o uso do nome
print('Visualizações:', match['visualizacoes']) # note o uso do nome

Visualizações: 1525
Visualizações: 1525


Agora é necessário obter o `<nome da obra>`.
A heurística é que o `<nome da obra>` é todo o texto entre `. ` e ` by`.
Portanto, como é possível fazer a captura de texto direto, podemos usar essas sequências para delimitar o nome da obra.

Apenas um detalhe: `.` na expressão regular, sinigifa qualquer caractére, exceto o caractére de nova linha (`\n`).
Portanto, se desejamos obter o caractére `.`, é preciso usar `\.`.

In [10]:
match = re.search('\. (\w+)', lista_top_10[0])
print(match.group(1))
print(match[1])

match = re.search('\.\s(\w+)', lista_top_10[0]) # note o uso de \s para indicar um espaço em branco
print(match.group(1))
print(match[1])

The
The
The
The


`\w` captura qualquer litra, equivalente a `[a-z]`.

Porém, no caso acima precisamos capturar um grupo com: letras, espaços e pontuação.

Existem algumas formas para fazer isso, vamos começar explorando `[]`.

A notação de `[]` em expressões regulares, permite definir opções.
Por exemplo, `[01]` captura `0` ou `1`

In [11]:
match = re.search('[01]', '1256')
print(match[0])
match = re.search('[01]', '2516')
print(match[0])

match = re.search('[01]', '0256')
print(match[0])
match = re.search('[01]', '2506')
print(match[0])

match = re.search('[01]', '256')
print(match is None)
match = re.search('[01]', '256')
print(match is None)

match = re.search('[at]', 'abv')
print(match[0])
match = re.search('[at]', 'abv')
print(match[0])

match = re.search('[at]', 'tbv')
print(match[0])
match = re.search('[at]', 'tbv')
print(match[0])

match = re.search('[at]', 'bv')
print(match is None)
match = re.search('[at]', 'bv')
print(match is None)

match = re.search('[a0]', 'a12')
print(match[0])
match = re.search('[a0]', '0bc')
print(match[0])

1
1
0
0
True
True
a
a
t
t
True
True
a
0


Outra opção, é que `[]` permite definir sequências.
Por exemplo, `[0-9]` (números entre 0 e 9) ou `[a-z]` (todas as letras do alfabeto).

Portanto, `\d` é similar à `[0-9]` e `\w` é similar à `[a-z]`.

Assim, podemos usar `[]` para definir uma expressão que permite pegar o `<nome da obra>`

In [12]:
match = re.search('\. ([a-zA-Z,:;.\- ]+) by', lista_top_10[0])
print(match[1])

# note que [a-z] é diferente de [A-Z]
# outra opção é fazer a regex case sensitive

match = re.search('\. ([a-z,:;.\- ]+) by', lista_top_10[0], flags=re.IGNORECASE)
print(match[1])

# note que . dentro de [] não significa qualquer caractére, mas exatamente .
# ou seja, fora de [] é necessário \. e dentro de [] apenas . é suficiente

The Works of Edgar Allan Poe, The Raven Edition
The Works of Edgar Allan Poe, The Raven Edition


Finalmente, podemos capturar o nome do autor com uma expressão similar ao `<nome da obra>`.

In [13]:
match = re.search('by ([a-zA-Z\s]+) \(', lista_top_10[0])
print(match[1])

Edgar Allan Poe


Entretanto, note que o quinto item da lista não possui autor `5. Beowulf: An Anglo-Saxon Epic Poem (658)`.

Para isso, podemos usar o operador `?` para indicar que algo é opcional.

Outra opção seria `{0, 1}` para indicar que algo pode ocorrer 0 ou 1 vez.

In [14]:
match = re.search('by ([a-zA-Z\s]+) \(', lista_top_10[4])
print(match is None)

match = re.search('(by ([a-zA-Z\s]+))? \(', lista_top_10[4])
print(match is None)
print(match[1]) # by <nome do autor>
print(match[2]) # <nome do autor>

match = re.search('(by ([a-zA-Z\s]+))? \(', lista_top_10[1])
print(match is None)
print(match[1]) # by <nome do autor>
print(match[2]) # <nome do autor>

True
False
None
None
False
by Jane Austen
Jane Austen


No exemplo acima podemos perceber que existe um grupo que não deveria estar ali.

Esse não é um grupo que queremos extrair, apenas um agrupamento para colocar o autor como opcional.

Para isso podemos utilizar a notação de _non capturing groups_ `?:`

In [15]:
match = re.search('(?:by ([a-zA-Z\s]+))? \(', lista_top_10[4]) # note o ?: no ínicio do grupo que desejamos descartar
print(match is None)
print(match[1]) # <nome do autor>

match = re.search('(?:by ([a-zA-Z\s]+))? \(', lista_top_10[1])
print(match is None)
print(match[1]) # <nome do autor>

False
None
False
Jane Austen


Agora podemos ver uma expressão completa para capturar todas as informações em cada linha.

In [16]:
regex = r'(?P<posicao>\d+)\. (?P<obra>[a-zA-Z,:;.\- ]+?) (?:by (?P<autor>[a-zA-Z ]+) )?\((?P<visualizacoes>\d+)\)'
match = re.search(regex, lista_top_10[0])
print(match['posicao'])
print(match['obra'])
print(match['autor'])
print(match['visualizacoes'])

# opção em multiplas linhas

match = re.search(
    r'(?P<posicao>\d+)\. '
    r'(?P<obra>[a-zA-Z,:;.\- ]+) '
    r'(?:by (?P<autor>[a-zA-Z ]+) )'
    r'\((?P<visualizacoes>\d+)\)',
    lista_top_10[0]
)
print(match['posicao'])
print(match['obra'])
print(match['autor'])
print(match['visualizacoes'])

1
The Works of Edgar Allan Poe, The Raven Edition
Edgar Allan Poe
1525
1
The Works of Edgar Allan Poe, The Raven Edition
Edgar Allan Poe
1525


Note que a expressão de `<obra>` mudou de `(?P<obra>[a-zA-Z,:;.\- ]+)` para `(?P<obra>[a-zA-Z,:;.\- ]+?)`.

A `?` após o quantificador `+` muda o modo de _greedy_ para _lazy_.
* _greedy_ : captura a maior sequência possível
* _lazy_ : para a captura assim que possível

O que acontece se removermos a `?`?

Uma alternativa para também seria utilizar o operador `|` com grupos `()`. Assim, `(a|b)` indica `a` ou `b`, sendo que `a` e `b` podem ser expressões arbitrárias e não apenas caracatéres. Portanto, é possível escrever `([a-z]|[0-9])` para capturar uma letra ou um número qualquer.

In [17]:
match = re.search(
    r'(?P<posicao>\d+)\. '
    r'(?:'
        r'(?P<obra1>[a-zA-Z,:;.\- ]+) by (?P<autor>[a-zA-Z ]+)'
        r'|'
        r'(?P<obra2>[a-zA-Z,:;.\- ]+)'
    r')'
    r'\((?P<visualizacoes>\d+)\)',
    lista_top_10[0]
)
print(match['posicao'])
print(match['obra1'])
print(match['autor'])
print(match['visualizacoes'])

match = re.search(
    r'(?P<posicao>\d+)\. '
    r'(?:'
        r'(?P<obra1>[a-zA-Z,:;.\- ]+) by (?P<autor>[a-zA-Z ]+)'
        r'|'
        r'(?P<obra2>[a-zA-Z,:;.\- ]+)'
    r')'
    r'\((?P<visualizacoes>\d+)\)',
    lista_top_10[4]
)
print(match['posicao'])
print(match['obra2'])
print(match['autor'])
print(match['visualizacoes'])

1
The Works of Edgar Allan Poe, The Raven Edition
Edgar Allan Poe 
1525
5
Beowulf: An Anglo-Saxon Epic Poem 
None
658


Porém, note como a expressão é mais complexa e foi necessário introduzir `<obra1>` e `<obra2>`.

De forma geral, o operador `|` é muito útil, só é necessário utilizá-lo no momento adequado.

Agora podemos mapear uma lista de obras para as informações relevantes..

In [18]:
def extrair_informacoes(texto):
    regex = r'(?P<posicao>\d+)\. (?P<obra>[a-zA-Z,:;.\- ]+?) (?:by (?P<autor>[a-zA-Z ]+) )?\((?P<visualizacoes>\d+)\)'
    match = re.search(regex, texto)
    return (
        int(match['posicao']),
        match['obra'],
        match['autor'],
        int(match['visualizacoes'])
    )

dados = map(extrair_informacoes, lista_top_10)
for d in dados:
    print(d)

(1, 'The Works of Edgar Allan Poe, The Raven Edition', 'Edgar Allan Poe', 1525)
(2, 'Pride and Prejudice', 'Jane Austen', 1302)
(3, 'Frankenstein; Or, The Modern Prometheus', 'Mary Wollstonecraft Shelley', 995)
(4, 'A Modest Proposal', 'Jonathan Swift', 674)
(5, 'Beowulf: An Anglo-Saxon Epic Poem', None, 658)
(6, 'Moby Dick; Or, The Whale', 'Herman Melville', 600)
(7, 'The Strange Case of Dr. Jekyll and Mr. Hyde', 'Robert Louis Stevenson', 560)
(8, 'The Adventures of Sherlock Holmes', 'Arthur Conan Doyle', 547)
(9, 'A Tale of Two Cities', 'Charles Dickens', 491)
(10, 'Ulysses', 'James Joyce', 481)


## Exercício


In [22]:
# escreva uma expressão regular para extrair os componentes das datas abaixo

datas = [
    '28/10/2019',
    '14/07/2019',
    '01/10/2018',
    '05/12/2017'
]

regex = r'' # escreva a sua regex aqui
match = re.search(regex, datas[0])
print(match[1], match[2], match[3]) # 28 10 2019

In [ ]:
# executando para todos os casos
for data in datas:
    match = re.search(regex, data)
    print(match[1], match[2], match[3])

In [ ]:
# altere a regex para considerar os seguintes casos
datas = [
    '2/10/2019',
    '14/7/2019',
    '01/10/18'
]

regex = r'' # escreva a sua regex aqui
match = re.search(regex, datas[0])
print(match[1], match[2], match[3]) # 2 10 2019

In [ ]:
# executando para todos os casos
for data in datas:
    match = re.search(regex, data)
    print(match[1], match[2], match[3])

In [ ]:
# altere a regex para considerar os seguintes casos
datas = [
    'Blumenau, 28 de Outubro de 2019',
    'Indaial, 9 de setembro de 2018',
]

regex = r'' # escreva a sua regex aqui
match = re.search(regex, datas[0])
print(match[1], match[2], match[3], match[4]) # Blumenau 28 Outubro 2019

In [ ]:
# executando para todos os casos
for data in datas:
    match = re.search(regex, data)
    print(match[1], match[2], match[3], match[4])